In [1]:
import pandas as pd

In [2]:
df = pd .read_csv("spam_classification/spam.csv",encoding="ISO-8859-1")
df = df[['v1','v2']]
df.columns = ['labels','message']
df

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [4]:
stemmer = PorterStemmer()
corpus = []

for i in range(len(df)):
    review = re.sub('[^a-zA-z]'," ",df['message'][i])
    review = review.lower()
    review = review.split()
    words = [stemmer.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = " ".join(words)
    corpus.append(review)

In [5]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breather

In [24]:
# Dependent feature
y = pd.get_dummies(df['labels'])
y = y.iloc[:, 0].values
print(y)

[ True  True False ...  True  True  True]


In [25]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus,y,test_size=0.20, random_state=42)

In [26]:
# Applying bow
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=2500, ngram_range=(1,2))

In [27]:
#Independent feature
X_train = bow.fit_transform(X_train).toarray()
X_test = bow.transform(X_test).toarray()

In [29]:
bow.vocabulary_

{'still': np.int64(1976),
 'mom': np.int64(1333),
 'check': np.int64(328),
 'yo': np.int64(2489),
 'half': np.int64(886),
 'bank': np.int64(139),
 'issu': np.int64(1030),
 'strong': np.int64(2001),
 'buy': np.int64(229),
 'pick': np.int64(1568),
 'member': np.int64(1274),
 'per': np.int64(1549),
 'give': np.int64(795),
 'second': np.int64(1808),
 'chanc': np.int64(315),
 'play': np.int64(1587),
 'lt': np.int64(1213),
 'gt': np.int64(860),
 'lt gt': np.int64(1215),
 'privat': np.int64(1643),
 'account': np.int64(9),
 'statement': np.int64(1970),
 'show': np.int64(1870),
 'un': np.int64(2218),
 'redeem': np.int64(1709),
 'point': np.int64(1608),
 'call': np.int64(240),
 'identifi': np.int64(986),
 'code': np.int64(362),
 'expir': np.int64(642),
 'privat account': np.int64(1644),
 'account statement': np.int64(10),
 'statement show': np.int64(1971),
 'show un': np.int64(1872),
 'un redeem': np.int64(2219),
 'redeem point': np.int64(1710),
 'point call': np.int64(1609),
 'call identifi': n

In [31]:
# Using Naive bayes algo 
from sklearn.naive_bayes import MultinomialNB

In [32]:
model = MultinomialNB().fit(X_train,y_train)
y_pred = model.predict(X_test)

In [33]:
# Model Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [34]:
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9802690582959641
[[135  15]
 [  7 958]]
              precision    recall  f1-score   support

       False       0.95      0.90      0.92       150
        True       0.98      0.99      0.99       965

    accuracy                           0.98      1115
   macro avg       0.97      0.95      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [36]:
# For tfidf
# train test split
X_tf_train, X_tf_test, y_tf_train, y_tf_test = train_test_split(corpus,y,test_size=0.20, random_state=42)

In [16]:
# Creating tfidf model
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2500, ngram_range=(1,2))

In [37]:
X_tf_train = tfidf.fit_transform(X_tf_train).toarray()
X_tf_test = tfidf.transform(X_tf_test).toarray()

In [38]:
tfidf_model = MultinomialNB().fit(X_tf_train,y_tf_train)
y_pred_tf = tfidf_model.predict(X_tf_test)

In [39]:
print(accuracy_score(y_tf_test,y_pred_tf))
print(confusion_matrix(y_tf_test,y_pred_tf))
print(classification_report(y_tf_test,y_pred_tf))

0.9766816143497757
[[124  26]
 [  0 965]]
              precision    recall  f1-score   support

       False       1.00      0.83      0.91       150
        True       0.97      1.00      0.99       965

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.95      1115
weighted avg       0.98      0.98      0.98      1115

